In [2]:
import datetime
now = datetime.datetime.now()
print ("Current date and time : ")
print (now.strftime("%Y-%m-%d %H:%M:%S"))

Current date and time : 
2022-11-07 23:39:35


In [1]:
import importlib
import pandas as pd
import numpy as np
import torch
import nltk
import documentSummaries
from topicModel import TopicModel as tm
from documentSummaries import DocumentSummaries
from keras.layers import Input, Flatten, Dense
from keras_preprocessing.sequence import pad_sequences
from keras import Model
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, TFBertForPreTraining
from copy import deepcopy
importlib.reload(documentSummaries)

<module 'documentSummaries' from 'C:\\Users\\clvnt\\OneDrive\\Desktop\\BT4222\\Python Notebook files\\Project\\BERT\\documentSummaries.py'>

In [2]:
df = pd.read_csv('final_compiled_final.csv')
df.head()

C:\Users\clvnt\AppData\Local\Temp\ipykernel_21444\713654814.py:1: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('final_compiled_final.csv')


,Unnamed: 0,id,name,artists,artist_ids,track_number,danceability,energy,key,loudness,...,liveness,valence,tempo,explicit,duration_ms,date,release_date,genres,lyrics,target
0,0,0gplL1WMoJ6iYaPgMCL0gX,Easy On Me,['Adele'],['4dpARuHxo51G3z768sgnrY'],1,0.604,0.366,5,-7.519,...,0.1330,0.130,141.981,False,224695,6/11/2021,NaN,"['pop soul', 'pop', 'british soul', 'uk pop']",. There ain't no gold in this river. That I've...,1
1,1,4BI4iNZigfF4SUq13TcFPh,INDUSTRY BABY - EXTENDED,"['Lil Nas X', 'Jack Harlow']","['7jVv8c5Fj3E9VhNjxT4snq', '2LIk90788K0zvyj2JJ...",2,0.732,0.708,10,-6.775,...,0.0528,0.883,149.988,True,227200,6/11/2021,NaN,"['deep underground hip hop', 'lgbtq+ hip hop',...",". (D-D-Daytrip took it to ten, hey). Baby back...",1
2,2,58UKC45GPNTflCN6nwCUeF,Fancy Like,['Walker Hayes'],['7sKxqpSqbIzphAKAhrqvlf'],6,0.647,0.765,1,-6.459,...,0.3150,0.855,79.994,False,161853,6/11/2021,NaN,"['modern country rock', 'contemporary country'...",". Ayy. My girl is bangin', she's so low mainte...",1
3,3,3rmo8F54jFF8OgYsqTxm5d,Bad Habits,['Ed Sheeran'],['6eUKZXaKkcviH0Ku9w2n3V'],4,0.807,0.893,11,-3.745,...,0.3660,0.537,126.011,False,230747,6/11/2021,NaN,"['pop', 'uk pop']",". (One, two, three, four). Ooh, ooh. . Every t...",1
4,4,50nfwKoDiSYg8zOCREWAm5,Shivers,['Ed Sheeran'],['6eUKZXaKkcviH0Ku9w2n3V'],2,0.788,0.859,2,-2.724,...,0.0424,0.822,141.020,False,207853,6/11/2021,NaN,"['pop', 'uk pop']",. I took an arrow to the heart. I never kissed...,1


In [3]:
df_focus = df[['id','name','lyrics','target']]
df_focus.head()

,id,name,lyrics,target
0,0gplL1WMoJ6iYaPgMCL0gX,Easy On Me,. There ain't no gold in this river. That I've...,1
1,4BI4iNZigfF4SUq13TcFPh,INDUSTRY BABY - EXTENDED,". (D-D-Daytrip took it to ten, hey). Baby back...",1
2,58UKC45GPNTflCN6nwCUeF,Fancy Like,". Ayy. My girl is bangin', she's so low mainte...",1
3,3rmo8F54jFF8OgYsqTxm5d,Bad Habits,". (One, two, three, four). Ooh, ooh. . Every t...",1
4,50nfwKoDiSYg8zOCREWAm5,Shivers,. I took an arrow to the heart. I never kissed...,1


In [4]:
topicModel = tm(num_topics=30,max_doc_length=31971,min_doc_length=0)
compiled_lyrics_list = []
for i in df_focus['lyrics']:
    compiled_lyrics_list.append(i)

In [5]:
topicModel.fit(compiled_lyrics_list)


number of low frequency tokens pruned = 218,011
min_word_count = 20, top_most_common_words = 10
number of high frequency tokens pruned = 10
tokens = 57,279 rows
text pre-processing is complete

computing LDA...
computing dominant topics...


In [6]:
overall_docs_group = []
for i in df_focus['lyrics']:
    sentence_detector = nltk.data.load('tokenizers/punkt/english.pickle')
    sentences = sentence_detector.tokenize(i)
    doc_group = []
    for j in range(0,len(sentences),4):
        doc_group.append(' '.join(sentences[j-4:j]))
    doc_group.pop(0)
    overall_docs_group.append(doc_group)

In [8]:
chosen_verses_for_each_song = []
for i in overall_docs_group:
    doc_sum = DocumentSummaries(topicModel,num_dominant_topics=30,number_of_sentences=3)
    doc_sum.summarize(i)
    result = doc_sum.return_possible_sentences()
    chosen_verses_for_each_song.append(result[:3])

In [11]:
import pickle
with open("chosen_verses", "wb") as fp:
    pickle.dump(chosen_verses_for_each_song, fp)

In [12]:
with open("chosen_verses", "rb") as fp:   # Unpickling
    test = pickle.load(fp)

In [14]:
print(test[0])

['Had no time to choose what I chose to do.', "You can't deny how hard I've tried.", 'I was still a child.']
